<a href="https://colab.research.google.com/github/FacuD/spotify-api-ETL/blob/main/SPOTIFY_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlalchemy
import pandas as pd 
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3

In [ ]:
DATABASE_LOCATION = "sqlite:///my_played_tracks.sqlite"
USER_ID = "" # your Spotify username 
TOKEN = "" # your Spotify API token

# Generate your token here:  https://developer.spotify.com/console/get-recently-played/
# Note: You need a Spotify account (can be easily created for free)

def check_data(df: pd.DataFrame) -> bool:
    # Check if the data is empty
    if df.empty:
        print("No songs downloaded.")
        return False
    # Check PK
    # print(list(df['played_at']))
    if not pd.Series(df['played_at']).is_unique:
        raise Exception("PK is violated.")
    # Check for nulls
    if df.isnull().values.any():
        raise Exception("Null/s are present.")

    return True


if __name__ == "__main__":

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {TOKEN}"
    }

    r = requests.get(
        "https://api.spotify.com/v1/me/player/recently-played", headers=headers)
    data = r.json()

    # Extract
    song_names = []
    artist_names = []
    played_at_list = []
    timestamps = []
    for song in data["items"]:
        song_names.append(song["track"]["name"])
        artist_names.append(song["track"]["album"]["artists"][0]["name"])
        played_at_list.append(song["played_at"][0:-5].replace('T',' '))
        timestamps.append(song["played_at"][0:10])
    songDict = {
        "song_name": song_names,
        "artist_name": artist_names,
        "played_at": played_at_list,
        "timestamp": timestamps
    }
    
    songDf = pd.DataFrame(
        songDict, columns=["song_name", "artist_name", "played_at", "timestamp"])
    # Transform (Validations)
    if check_data(songDf):
        print("Data is valid.")

    # Load
    engine = sqlalchemy.create_engine(DATABASE_LOCATION)
    conn = sqlite3.connect("my_played_tracks.sqlite")
    cursor = conn.cursor()

    query = """
    CREATE TABLE IF NOT EXISTS my_played_tracks(
      song_name VARCHAR(200),
      artist_name VARCHAR(200),
      played_at VARCHAR(200),
      timestamp VARCHAR(200),
      CONSTRAINT primary_key_constraint PRIMARY KEY (played_at)
    )
    """

    cursor.execute(query)
    print("Database opened")

    try:
      songDf.to_sql("my_played_tracks", engine, index=False, if_exists='append')
    except:
      print("Data already exists in the database")
    
    cursor.close()
    print("Database closed")

  

Data is valid.
Database opened
Data already exists in the database
Database closed
